In [1]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline
# %matplotlib notebook

In [2]:
import sys
import os

sys.path.append('../tools')

In [3]:
import numpy as np
import matplotlib

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch
from tqdm import tqdm
import sklearn

import torchvision.transforms as T
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
import pytorch_lightning.callbacks as pl_callbacks

matplotlib.use("nbAgg")

import data_utility

from itertools import groupby
from tqdm import tqdm

In [4]:
import matplotlib.pyplot as plt

In [5]:
data_dir = "../../../user_data/"
log_folder_root = '../../../user_data/logs/'
ckpt_folder_root = '../../../user_data/checkpoints/'

In [6]:
dir_list = os.listdir(data_dir+'rns_data')
patientIDs = [s for s in dir_list for type_string in ['HUP', 'RNS'] if type_string in s.upper()]

In [7]:
os.listdir(data_dir+'rns_cache')

['HUP047.npy',
 'HUP059.npy',
 'HUP084.npy',
 'HUP096.npy',
 'HUP101.npy',
 'HUP108.npy',
 'HUP109.npy',
 'HUP121.npy',
 'HUP127.npy',
 'HUP128.npy',
 'HUP129.npy',
 'HUP131.npy',
 'HUP136.npy',
 'HUP137.npy',
 'HUP143.npy',
 'HUP147.npy',
 'HUP153.npy',
 'HUP156.npy',
 'HUP159.npy',
 'HUP182.npy',
 'HUP192.npy',
 'HUP197.npy',
 'HUP199.npy',
 'HUP205.npy',
 'RNS021.npy',
 'RNS022.npy',
 'RNS026.npy',
 'RNS029.npy']

In [8]:
data_import = data_utility.read_files(path = data_dir+'rns_data', path_data = data_dir+'rns_raw_cache',patientIDs=patientIDs[10:20], annotation_only = False, verbose=True)
ids = list(data_import.keys())

100%|██████████| 10/10 [00:10<00:00,  1.06s/it]


In [9]:
window_len = 1
stride = 1
concat_n = 4

for id in tqdm(ids):
    data_import[id].set_window_parameter(window_length=window_len, window_displacement=stride)
    data_import[id].set_concatenation_parameter(concatenate_window_n=concat_n)
    data_import[id].get_windowed_data(data_import[id].catalog["Event Start idx"],data_import[id].catalog["Event End idx"])
    data_import[id].normalize_windowed_data()
    _, concatenated_data = data_import[id].get_concatenated_data(data_import[id].windowed_data, arrange='channel_stack')
    concatenated_data = concatenated_data[::3]
    print(concatenated_data.shape)

    np.save(data_dir+'rns_cache/'+ id +'.npy',concatenated_data)


  0%|          | 0/10 [00:00<?, ?it/s]

bla
(54777, 249, 36)


 10%|█         | 1/10 [00:21<03:12, 21.34s/it]

(123528, 249, 36)


 20%|██        | 2/10 [01:12<05:13, 39.16s/it]

...

 80%|████████  | 8/10 [05:47<01:31, 45.85s/it]

(13585, 249, 36)


 90%|█████████ | 9/10 [05:53<00:33, 33.33s/it]

bla
(77735, 249, 36)


100%|██████████| 10/10 [06:28<00:00, 38.86s/it]


In [10]:
from models.rns_dataloader import RNS_Raw
# unlabeled_dataset = RNS_Raw(file_list, transform=True,astensor = False)
from lightly.data import SwaVCollateFunction

dir_list = os.listdir(data_dir+'rns_cache')
# file_list = ['HUP084.npy','HUP131.npy','HUP096.npy']
# if self.current_epoch == 0:
#     file_list = ['HUP101.npy']
file_list = ['RNS026.npy', 'HUP159.npy', 'HUP129.npy', 'HUP096.npy']
unlabeled_dataset = RNS_Raw(file_list, transform=True, astensor=False)

collate_fn = SwaVCollateFunction(gaussian_blur=0, hf_prob=0, vf_prob=0, rr_prob=0, cj_prob=0,
                                 random_gray_scale=0, normalize={'mean': [0, 0, 0], 'std': [1, 1, 1]})

dataloader = torch.utils.data.DataLoader(
    unlabeled_dataset,
    batch_size=340,
    collate_fn=collate_fn,
    shuffle=True,
    drop_last=True,
    num_workers=2
)

data loaded


In [1]:
from models.SwaV import SwaV
from lightly.data import SwaVCollateFunction
#
model = SwaV()

# model.load_from_checkpoint()

# collate_fn = SwaVCollateFunction(gaussian_blur = 0, hf_prob = 0,vf_prob = 0,rr_prob=0,cj_prob=0,random_gray_scale=0, normalize={'mean':[0, 0, 0], 'std':[1, 1, 1]})
#
# dataloader = torch.utils.data.DataLoader(
#     unlabeled_dataset,
#     batch_size=256,
#     collate_fn=collate_fn,
#     shuffle=True,
#     drop_last=True,
#     num_workers=4
# )
accelerator = "gpu" if torch.cuda.is_available() else "cpu"

checkpoint_callback = pl_callbacks.ModelCheckpoint(monitor='swav_loss',filename='rns_swav-{epoch:02d}-{swav_loss:.5f}', save_last=True, save_top_k=-1, dirpath=ckpt_folder_root + 'rns_swav_34_all')
csv_logger = pl_loggers.CSVLogger(log_folder_root, name="rns_swav_34_all")

trainer = pl.Trainer(logger=csv_logger, max_epochs=150, callbacks=[checkpoint_callback],accelerator='gpu', devices=1,precision=16,reload_dataloaders_every_n_epochs=1)

NameError: name 'torch' is not defined

In [12]:
trainer.fit(model=model,train_dataloaders=dataloader,ckpt_path=ckpt_folder_root + 'rns_swav_34_all/rns_swav-epoch=21-swav_loss=3.09745.ckpt')

C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:616: UserWarning: Checkpoint directory C:\Users\Patrick Xu\Desktop\RNS_Annotation-Pipeline\user_data\checkpoints\rns_swav_34_all exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
Restoring states from the checkpoint path at ../../../user_data/checkpoints/rns_swav_34_all/rns_swav-epoch=21-swav_loss=3.09745.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type               | Params
-------------------------------------------------------
0 | backbone        | Sequential         | 21.3 M
1 | projection_head | SwaVProjectionHead | 328 K 
2 | prototypes      | SwaVPrototypes     | 66.0 K
3 | queues          | ModuleList         | 0     
4 | criterion       | SwaVLoss           | 0     
-------------------------------------------------------
21.7 M    Trainable params
0         Non-trainable para

Training: 2180it [00:00, ?it/s]

C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
